In [ ]:
#For reading *_SPT.txt files, reareange them and same as NC files in same name
import pandas as pd
import numpy as np
import xarray as xr
from datetime import datetime as dt
import glob
import os
path1=os.getcwd()
dirc=[]
for fol in next(os.walk('.'))[1]:
    if not fol.startswith('.'):
        dirc.append(fol)
for folder in dirc:
    path="{}/{}/*.txt".format(path1,folder)
    #path = "*.txt"     #Give path of SPT files here
    files=glob.glob(path)
    #print(path,files)
    for spt_file in files:
        result=[]
        time=[]
        file_name=spt_file[:-4] #Extracting file name without extension (*.txt)
        with open(spt_file) as file:
            for line in file:
                dat=line.rstrip()
                res = dat.split()
                res=[ x for x in res if "m" not in x] # removing Charcter "Time Stamp" from lines
                if len(res)==10:  # Detecting line conatains date
                    date=res[0]   #Storing date on other variable
                    res=res[1:]
                    time.append(date)
                    result.append(res)
                else:
                    result.append(res)
        result=pd.DataFrame(result,columns=["Frequency","S(f)","Dir(f)","Spr(f)",
                                            "Skew(f)","Kurt(f)","K","Lat","Lon"]) #creating dataframe using arranged data for one file 
        time = np.array(time, dtype='datetime64') #converting time axis to datetime format for creating coordinates in NC file 
        Frequency=result["Frequency"].iloc[0:64] # Extracting frequency from data 
        Frequency= np.array(Frequency, dtype='float')
        Spect=np.array(result["S(f)"],dtype="float").reshape(len(time),64) 
        Dir=np.array(result["Dir(f)"],dtype="float").reshape(len(time),64)
        Spr=np.array(result["Spr(f)"],dtype="float").reshape(len(time),64)
        Skew=np.array(result["Skew(f)"],dtype="float").reshape(len(time),64)
        Kurt=np.array(result["Kurt(f)"],dtype="float").reshape(len(time),64)
        K=np.array(result["K"],dtype="float").reshape(len(time),64)
        Lat=np.array(result["Lat"],dtype="float").reshape(len(time),64)
        Lon=np.array(result["Lon"],dtype="float").reshape(len(time),64)
        ds = xr.Dataset(
            coords=dict(
                    time=(["time"], time),
                    Frequency=(["f"], Frequency),
                ),
            data_vars=dict(
                    S=(["time","f"], Spect),
                    Dir=(["time","f"], Dir),
                    Spr=(["time","f"], Spr),
                    Kurt=(["time","f"], Kurt),
                    Skew=(["time","f"], Skew),
                    K=(["time","f"], K),
                )
            )
        dtnow=dt.now().strftime("%Y-%m-%dT%H:%M:%SZ")  
        ##Setting attributes
        ds.attrs["date_created"]=dtnow   #storing created dates in NC file
        ds.time.attrs["Units"] = "Hour" 
        ds.Frequency.attrs["Units"]="Hz"
        ds.S.attrs["Units"] = "Hz/m2"
        ds.S.attrs["Long_Name"] = "Spectral_Energy"
        ds.Spr.attrs["Long_Name"]="Directional spread"
        ds.Spr.attrs["Units"]="Degree"
        ds.Kurt.attrs["Long_Name"]="Kurtosis of the directional distribution"
        ds.Skew.attrs["Long_Name"]="Skewness of the directional distribution"
        ds.Dir.attrs["Long_Name"]="Wave direction"
        ds.Dir.attrs["units"] = "degree_north"
        ds.to_netcdf("{}.nc".format(file_name))